<a href="https://colab.research.google.com/github/raj166/Machine-Learning-Basic-models/blob/main/Multi_class_Classification_on_MB_profile_v0_1%20with%20keras%20tuner%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv("AudienceResultsSim_subFactor.csv", index_col=0)
df.head()

,BirthDate,Sign,MB Profile,age,faminc,race,marstat,educ,employ,gender,...,Gentleness,Flexibility,Organization,Diligence,Perfectionism,Prudence,Aesthetic Appreciation,Inquisitiveness,Creativity,Unconventionality
0,7/7,Cancer,INFJ,28,"$60,000 - $69,999",Middle Eastern,Divorced,2-Year,Homemaker,Female,...,-0.6,-2.4,-0.2,0.1,1.8,-1.2,-0.4,0.3,0.7,-0.7
1,8/25,Virgo,ESTJ,20,"$350,000 - $499,999",Hispanic,Widowed,Some College,Homemaker,Female,...,0.0,-1.2,0.4,-0.5,1.5,-2.1,-0.3,0.5,0.4,0.0
2,11/4,Scorpio,INFJ,20,"$150,000 - $199,999",Middle Eastern,Domestic/Civil Partnership,HS Grad,Full Time,Female,...,-1.2,-1.9,0.4,-0.7,1.6,-0.5,-0.4,0.9,0.9,0.3
3,7/6,Cancer,INFJ,36,"$250,000 - $349,999",Native American,Divorced,2-Year,Temporarily Laid Off,Female,...,-1.2,-1.4,0.5,0.0,1.9,-1.0,0.0,1.3,0.6,0.5
4,5/4,Taurus,ISTJ,48,"$500,000 or more",White,Divorced,4-Year,Other,Female,...,-1.2,-1.4,-0.2,-0.8,2.1,-1.5,-0.9,0.0,0.7,-0.4


In [ ]:
df.drop(labels=["Latitude","Longitude"], axis=1, inplace=True)

In [ ]:
df.head()

,BirthDate,Sign,MB Profile,age,faminc,race,marstat,educ,employ,gender,...,Gentleness,Flexibility,Organization,Diligence,Perfectionism,Prudence,Aesthetic Appreciation,Inquisitiveness,Creativity,Unconventionality
0,7/7,Cancer,INFJ,28,"$60,000 - $69,999",Middle Eastern,Divorced,2-Year,Homemaker,Female,...,-0.6,-2.4,-0.2,0.1,1.8,-1.2,-0.4,0.3,0.7,-0.7
1,8/25,Virgo,ESTJ,20,"$350,000 - $499,999",Hispanic,Widowed,Some College,Homemaker,Female,...,0.0,-1.2,0.4,-0.5,1.5,-2.1,-0.3,0.5,0.4,0.0
2,11/4,Scorpio,INFJ,20,"$150,000 - $199,999",Middle Eastern,Domestic/Civil Partnership,HS Grad,Full Time,Female,...,-1.2,-1.9,0.4,-0.7,1.6,-0.5,-0.4,0.9,0.9,0.3
3,7/6,Cancer,INFJ,36,"$250,000 - $349,999",Native American,Divorced,2-Year,Temporarily Laid Off,Female,...,-1.2,-1.4,0.5,0.0,1.9,-1.0,0.0,1.3,0.6,0.5
4,5/4,Taurus,ISTJ,48,"$500,000 or more",White,Divorced,4-Year,Other,Female,...,-1.2,-1.4,-0.2,-0.8,2.1,-1.5,-0.9,0.0,0.7,-0.4


In [ ]:
Processed_df = df.copy(deep=True)

In [ ]:
!pip install category-encoders

In [ ]:
import category_encoders as ce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
Processed_df.drop(labels=['BirthDate', "zipcode" , "Place Name" , "Census Region" ], axis = 1, inplace = True)
Processed_df.head()

,Sign,MB Profile,age,faminc,race,marstat,educ,employ,gender,State,...,Gentleness,Flexibility,Organization,Diligence,Perfectionism,Prudence,Aesthetic Appreciation,Inquisitiveness,Creativity,Unconventionality
0,Cancer,INFJ,28,"$60,000 - $69,999",Middle Eastern,Divorced,2-Year,Homemaker,Female,TX,...,-0.6,-2.4,-0.2,0.1,1.8,-1.2,-0.4,0.3,0.7,-0.7
1,Virgo,ESTJ,20,"$350,000 - $499,999",Hispanic,Widowed,Some College,Homemaker,Female,IN,...,0.0,-1.2,0.4,-0.5,1.5,-2.1,-0.3,0.5,0.4,0.0
2,Scorpio,INFJ,20,"$150,000 - $199,999",Middle Eastern,Domestic/Civil Partnership,HS Grad,Full Time,Female,OH,...,-1.2,-1.9,0.4,-0.7,1.6,-0.5,-0.4,0.9,0.9,0.3
3,Cancer,INFJ,36,"$250,000 - $349,999",Native American,Divorced,2-Year,Temporarily Laid Off,Female,FL,...,-1.2,-1.4,0.5,0.0,1.9,-1.0,0.0,1.3,0.6,0.5
4,Taurus,ISTJ,48,"$500,000 or more",White,Divorced,4-Year,Other,Female,AL,...,-1.2,-1.4,-0.2,-0.8,2.1,-1.5,-0.9,0.0,0.7,-0.4


In [ ]:
from sklearn.model_selection import train_test_split
y = Processed_df['MB Profile']
X = Processed_df.drop('MB Profile', axis =1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=69, stratify = y)

In [ ]:
y_test

7809         ISFP
4976         INFP
3909         ESTJ
9506         INFJ
3049         ESTJ
          ...    
1404         ESTJ
5052         INFP
6296         ISTJ
8218    ESFP/ENFP
5566         ENFJ
Name: MB Profile, Length: 2002, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

ord_enc = LabelEncoder()
ord_enc.fit(y_train)
y_train = ord_enc.transform(y_train)
y_test = ord_enc.transform(y_test)

In [ ]:
y_test

array([8, 6, 3, ..., 9, 2, 0])

In [ ]:
import category_encoders as ce
cbe_encoder = ce.cat_boost.CatBoostEncoder()
  
# Fit encoder and transform the features
cbe_encoder.fit(X_train, y_train)
X_train = cbe_encoder.transform(X_train)
X_test = cbe_encoder.transform(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
train_x = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
test_x = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [ ]:
X_train


,Sign,age,faminc,race,marstat,educ,employ,gender,State,Neighborhood Nickname,...,Gentleness,Flexibility,Organization,Diligence,Perfectionism,Prudence,Aesthetic Appreciation,Inquisitiveness,Creativity,Unconventionality
1150,6.995602,17,4.373614,4.128544,4.064626,4.104238,3.986838,4.128705,4.102734,4.019199,...,-0.6,-1.7,-0.2,-0.2,1.3,-1.2,-1.0,0.4,0.6,-0.3
6100,7.993834,48,4.080391,4.128544,4.144324,4.123782,4.047853,4.128705,3.791404,4.115192,...,0.1,-2.4,0.1,0.3,1.5,-0.9,0.6,0.8,0.5,-0.5
4244,1.004588,25,4.104317,4.154579,4.012080,4.123782,4.067168,4.128705,4.280448,2.705064,...,-1.1,-1.1,-0.5,-0.4,2.1,-1.2,0.2,0.8,0.2,0.6
8657,7.993834,42,4.080391,4.160850,4.064626,4.123782,4.110754,4.101150,4.237344,4.346132,...,-0.5,-2.0,0.2,0.1,2.3,-0.9,0.2,-0.1,1.0,0.7
9563,4.998613,32,4.094611,4.207045,4.197477,3.981689,4.029602,4.128705,4.080279,4.115192,...,-0.1,-2.0,0.1,-0.3,2.6,-2.0,0.5,0.7,0.1,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,6.995602,28,4.020312,4.185910,4.162704,4.098905,4.230986,4.128705,3.791404,3.278798,...,-0.6,-1.8,-0.2,-0.4,0.9,-0.9,-0.3,1.2,0.4,0.6
6755,0.005804,26,4.039126,4.185910,4.012080,3.981689,4.216917,4.128705,3.812367,4.115192,...,0.2,-2.0,-0.6,-0.3,1.5,-1.7,-0.3,-0.4,0.1,-0.5
3105,1.004588,26,4.017085,4.185910,4.012080,4.122866,4.029602,4.128705,4.032554,3.278798,...,-0.6,-2.6,-0.3,-0.5,1.6,-1.3,0.1,0.5,0.4,1.1
8569,4.998613,25,4.280880,4.154579,4.144324,4.104238,3.986838,4.101150,4.080279,3.074346,...,-1.0,-1.5,0.5,-0.1,2.3,-1.1,0.4,0.8,0.9,-0.2


In [ ]:
def f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, ReLU, LeakyReLU, ELU, Activation
import numpy as np
from keras import backend as K


f1 = tfa.metrics.F1Score(6, 'micro')

In [ ]:
model = Sequential()

model.add(Dense(input_dim = 32, units = 8, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('LeakyReLU'))
#model.add(Dense(units = 16, activation = 'leaky_relu', kernel_initializer='he_uniform'))
model.add(Dense(units = 10, activation = 'softmax', kernel_initializer='glorot_uniform'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = [f1])

In [ ]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 16, validation_split = 0.2)

Epoch 1/100
401/401 [==============================] - 3s 8ms/step - loss: 1.3145 - f1_score: 0.1953 - val_loss: 1.1761 - val_f1_score: 0.1952
Epoch 2/100
401/401 [==============================] - 2s 6ms/step - loss: 1.2836 - f1_score: 0.1953 - val_loss: 1.1256 - val_f1_score: 0.1952
Epoch 3/100
401/401 [==============================] - 2s 5ms/step - loss: 1.2446 - f1_score: 0.1953 - val_loss: 1.0640 - val_f1_score: 0.1952
Epoch 4/100
401/401 [==============================] - 2s 5ms/step - loss: 1.1829 - f1_score: 0.1953 - val_loss: 1.0458 - val_f1_score: 0.1952
Epoch 5/100
401/401 [==============================] - 2s 5ms/step - loss: 1.1575 - f1_score: 0.1953 - val_loss: 0.9867 - val_f1_score: 0.1952
Epoch 6/100
401/401 [==============================] - 2s 5ms/step - loss: 1.1438 - f1_score: 0.1953 - val_loss: 0.9708 - val_f1_score: 0.1952
Epoch 7/100
401/401 [==============================] - 2s 6ms/step - loss: 1.1293 - f1_score: 0.1953 - val_loss: 0.9368 - val_f1_score: 0.1952

In [ ]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 3ms/step - loss: 0.6283 - f1_score: 0.1953


[0.6283437013626099, 0.19525155425071716]

In [ ]:
# HYPERPARAMETER TUNING

!pip install keras-tuner --upgrade

     |████████████████████████████████| 133 kB 4.2 MB/s 


In [ ]:
import keras_tuner as kt
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [ ]:
def build_model(hp):
  model = Sequential()

  for i in range(hp.Int('Dense Layers', min_value = 1, max_value = 3)):

    model.add(Dense(
        hp.Int('units', min_value = 2, max_value = 16, step = 4),
        input_dim = 32,
         
        kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

  #model.add(Dense(
   #   256,
    #  activation = 'relu', kernel_initializer='he_uniform'
 # ))

  model.add(Dense(units = 10, activation = 'softmax', kernel_initializer='glorot_uniform'))

  #learning_rate = hp.Float('learning_rate', min_value = 1e-4, max_value = 1e-2, sampling = 'log')

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('lr', min_value = 1e-5, max_value = 1e-2, sampling = 'log')), 
      loss = 'sparse_categorical_crossentropy', metrics = [f1])

  return model


In [ ]:
tuner = kt.RandomSearch(
    build_model,
    kt.Objective('val_f1_score', direction = 'max'),
    max_trials=15, seed = 69)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [ ]:
tuner.search(X_train, y_train, epochs=5, batch_size = 16, validation_data=(X_test, y_test))

Trial 15 Complete [00h 00m 19s]
val_f1_score: 0.19525155425071716

Best val_f1_score So Far: 0.19525155425071716
Total elapsed time: 00h 04m 42s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(X_test, y_test)

63/63 [==============================] - 1s 7ms/step - loss: 2.4848 - f1_score: 0.1953


[2.484780788421631, 0.19525155425071716]

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                462       
                                                                 
 batch_normalization (BatchN  (None, 14)               56        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                210       
                                                                 
 batch_normalization_1 (Batc  (None, 14)               56        
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 14)                0

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
Dense Layers: 2
units: 14
lr: 9.359659238480389e-05
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 1
units: 6
lr: 0.0037947689254228217
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 2
units: 6
lr: 0.002496583565707553
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 3
units: 6
lr: 6.523389943576093e-05
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 2
units: 6
lr: 0.00030794170455443216
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 2
units: 6
lr: 0.002369069534544757
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 1
units: 6
lr: 1.750942941667167e-05
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 1
units: 2
lr: 0.00018328792004345589
Score: 0.19525155425071716
Trial summary
Hyperparameters:
Dense Layers: 3
units: 10
l